# Predictor de Precios de Casas

Este notebook te permite hacer predicciones de precios para casas nuevas usando un modelo ya entrenado.

## ¿Cuándo usar este notebook?

Usa este notebook cuando:
- Ya tienes un modelo entrenado guardado
- Quieres predecir precios de casas nuevas
- Necesitas hacer predicciones en lote (múltiples casas a la vez)

## Flujo de trabajo

1. **Cargar modelo entrenado**: Usar el modelo guardado previamente
2. **Preparar datos**: Crear un DataFrame con las características de las casas nuevas
3. **Hacer predicciones**: Usar el modelo para estimar precios
4. **Analizar resultados**: Revisar las predicciones obtenidas

## 1. Importar librerías

Importamos las librerías necesarias para:
- **pandas**: Manejar los datos de las casas nuevas
- **config**: Obtener la lista de features que espera el modelo

In [10]:
import pandas as pd
import sys
import os

# Añadir el directorio padre al path para poder importar config
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import FEATURES

## 2. Función de predicción

### ¿Qué hace esta función?

La función `predict_new_houses()` es muy versátil y acepta dos tipos de entrada:

#### Opción 1: Lista de listas
```python
casas = [
    [150, 3, 2, 10, 5],  # Casa 1
    [200, 4, 3, 5, 3]    # Casa 2
]
```

#### Opción 2: DataFrame de pandas
```python
casas = pd.DataFrame({
    'tamano_m2': [150, 200],
    'habitaciones': [3, 4],
    'banos': [2, 3],
    'edad_anos': [10, 5],
    'distancia_centro_km': [5, 3]
})
```

### Proceso interno:

1. **Validación**: Convierte los datos a DataFrame si es necesario
2. **Asegura orden correcto**: Usa las columnas definidas en FEATURES
3. **Predice**: Aplica el modelo a cada casa
4. **Formatea salida**: Muestra los precios en formato legible ($XXX,XXX.XX)

### Importante:

Los datos de entrada **DEBEN** tener las mismas características que los datos de entrenamiento:
- Mismas columnas
- Mismo orden
- Mismas unidades (ej: metros cuadrados, no pies cuadrados)

In [19]:
def predict_new_houses(modelo, casas_nuevas_data):
    """
    Predice precios de casas nuevas
    
    Args:
        modelo: Modelo entrenado
        casas_nuevas_data: Lista de listas o DataFrame con las características
    
    Returns:
        predicciones: Array con los precios predichos
    """
    # Convertir a DataFrame si es necesario
    if not isinstance(casas_nuevas_data, pd.DataFrame):
        casas_nuevas = pd.DataFrame(casas_nuevas_data, columns=FEATURES)
    else:
        casas_nuevas = casas_nuevas_data
    
    print("\n=== PREDICIENDO CASAS NUEVAS ===")
    print(casas_nuevas)
    
    predicciones = modelo.predict(casas_nuevas)
    
    print("\nPredicciones:")
    for i, precio in enumerate(predicciones):
        print(f"Casa {i+1}: ${precio:,.2f}")
    
    return predicciones

## 3. Cargar el modelo entrenado

Antes de hacer predicciones, necesitamos cargar el modelo que fue entrenado previamente.

### Requisito previo:

Debes haber ejecutado el notebook `model.ipynb` y guardado el modelo. Si no lo has hecho:
1. Abre `model.ipynb`
2. Ejecuta todas las celdas para entrenar el modelo
3. Asegúrate de que se guardó correctamente el archivo `.pkl`

In [12]:
import pickle
from config import MODEL_FILE

# Cargar el modelo entrenado
with open(MODEL_FILE, 'rb') as f:
    modelo = pickle.load(f)

print(f"✓ Modelo cargado exitosamente desde: {MODEL_FILE}")
print(f"\nCaracterísticas que espera el modelo: {FEATURES}")

✓ Modelo cargado exitosamente desde: c:\dev\repos-deep-learning\predictor-house\models\modelo_casas.pkl

Características que espera el modelo: ['tamano_m2', 'habitaciones', 'banos', 'edad_anos', 'distancia_centro_km']


## 4. Ejemplo 1: Predicción de una sola casa

### Escenario:

Quieres saber el precio de una casa con estas características:
- **Tamaño**: 150 m²
- **Habitaciones**: 3
- **Baños**: 2
- **Antigüedad**: 10 años
- **Distancia al centro**: 5 km

In [13]:
# Opción 1: Usando lista
casa_nueva = [[150, 3, 2, 10, 5]]

precio_predicho = predict_new_houses(modelo, casa_nueva)

print(f"\n📊 Resultado: La casa valdría aproximadamente ${precio_predicho[0]:,.2f}")


=== PREDICIENDO CASAS NUEVAS ===
   tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km
0        150             3      2         10                    5

Predicciones:
Casa 1: $385,369.32

📊 Resultado: La casa valdría aproximadamente $385,369.32


In [14]:
# Opción 2: Usando DataFrame (más legible)
casa_nueva_df = pd.DataFrame({
    'tamano_m2': [150],
    'habitaciones': [3],
    'banos': [2],
    'edad_anos': [10],
    'distancia_centro_km': [5]
})

precio_predicho = predict_new_houses(modelo, casa_nueva_df)

print(f"\n📊 Resultado: La casa valdría aproximadamente ${precio_predicho[0]:,.2f}")


=== PREDICIENDO CASAS NUEVAS ===
   tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km
0        150             3      2         10                    5

Predicciones:
Casa 1: $385,369.32

📊 Resultado: La casa valdría aproximadamente $385,369.32


## 5. Ejemplo 2: Predicción en lote (múltiples casas)

### Escenario:

Una inmobiliaria quiere estimar precios para 5 casas diferentes de su catálogo.

Esto es útil cuando:
- Tienes múltiples propiedades para valorar
- Quieres comparar precios de diferentes opciones
- Necesitas procesar un catálogo completo

In [15]:
# Crear datos de múltiples casas
varias_casas = pd.DataFrame({
    'tamano_m2': [120, 180, 200, 90, 250],
    'habitaciones': [2, 3, 4, 2, 5],
    'banos': [1, 2, 3, 1, 3],
    'edad_anos': [15, 8, 3, 20, 1],
    'distancia_centro_km': [8, 4, 2, 10, 1]
})

print("Casas a evaluar:")
print(varias_casas)

# Hacer predicciones
precios = predict_new_houses(modelo, varias_casas)

Casas a evaluar:
   tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km
0        120             2      1         15                    8
1        180             3      2          8                    4
2        200             4      3          3                    2
3         90             2      1         20                   10
4        250             5      3          1                    1

=== PREDICIENDO CASAS NUEVAS ===
   tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km
0        120             2      1         15                    8
1        180             3      2          8                    4
2        200             4      3          3                    2
3         90             2      1         20                   10
4        250             5      3          1                    1

Predicciones:
Casa 1: $262,378.44
Casa 2: $466,432.46
Casa 3: $570,679.82
Casa 4: $176,141.83
Casa 5: $742,642.54


## 6. Análisis de resultados

Podemos crear una tabla con los resultados para un análisis más fácil:

In [16]:
# Añadir predicciones al DataFrame
varias_casas['precio_predicho'] = precios

# Calcular precio por m²
varias_casas['precio_por_m2'] = varias_casas['precio_predicho'] / varias_casas['tamano_m2']

print("\n=== ANÁLISIS DE RESULTADOS ===")
print(varias_casas)

print("\n=== ESTADÍSTICAS ===")
print(f"Precio promedio: ${varias_casas['precio_predicho'].mean():,.2f}")
print(f"Precio mínimo: ${varias_casas['precio_predicho'].min():,.2f}")
print(f"Precio máximo: ${varias_casas['precio_predicho'].max():,.2f}")
print(f"\nPrecio promedio por m²: ${varias_casas['precio_por_m2'].mean():,.2f}/m²")


=== ANÁLISIS DE RESULTADOS ===
   tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km  \
0        120             2      1         15                    8   
1        180             3      2          8                    4   
2        200             4      3          3                    2   
3         90             2      1         20                   10   
4        250             5      3          1                    1   

   precio_predicho  precio_por_m2  
0    262378.436198    2186.486968  
1    466432.456946    2591.291427  
2    570679.817203    2853.399086  
3    176141.831998    1957.131467  
4    742642.539897    2970.570160  

=== ESTADÍSTICAS ===
Precio promedio: $443,655.02
Precio mínimo: $176,141.83
Precio máximo: $742,642.54

Precio promedio por m²: $2,511.78/m²


## 7. Ejemplo 3: Comparar escenarios

### ¿Cuánto vale más una casa renovada?

Comparemos dos casas idénticas, pero una tiene 20 años y otra fue recién renovada (0 años):

In [17]:
comparacion = pd.DataFrame({
    'tamano_m2': [150, 150],
    'habitaciones': [3, 3],
    'banos': [2, 2],
    'edad_anos': [20, 0],  # Diferencia principal
    'distancia_centro_km': [5, 5]
}, index=['Casa antigua', 'Casa renovada'])

precios_comparacion = predict_new_houses(modelo, comparacion)

diferencia = precios_comparacion[1] - precios_comparacion[0]
porcentaje = (diferencia / precios_comparacion[0]) * 100

print(f"\n💡 INSIGHT:")
print(f"Una casa renovada vale ${diferencia:,.2f} más ({porcentaje:.1f}% de incremento)")
print(f"\nCada año de antigüedad reduce el precio en aproximadamente ${diferencia/20:,.2f}")


=== PREDICIENDO CASAS NUEVAS ===
               tamano_m2  habitaciones  banos  edad_anos  distancia_centro_km
Casa antigua         150             3      2         20                    5
Casa renovada        150             3      2          0                    5

Predicciones:
Casa 1: $341,394.95
Casa 2: $429,343.69

💡 INSIGHT:
Una casa renovada vale $87,948.75 más (25.8% de incremento)

Cada año de antigüedad reduce el precio en aproximadamente $4,397.44


## 8. Tu turno: Predice tus propias casas

Usa esta celda para hacer tus propias predicciones:

In [ ]:
# Modifica estos valores con las características de tu casa
mi_casa = pd.DataFrame({
    'tamano_m2': ["???"],              # Tamaño en metros cuadrados
    'habitaciones': ["???"],           # Número de habitaciones
    'banos': ["???"],                  # Número de baños
    'edad_anos': ["???"],              # Antigüedad en años
    'distancia_centro_km': ["???"]    # Distancia al centro en km
})

# Hacer predicción
mi_precio = predict_new_houses(modelo, mi_casa)

print(f"\n🏠 Tu casa valdría aproximadamente: ${mi_precio[0]:,.2f}")

SyntaxError: invalid syntax (2369390571.py, line 3)

## 9. Consejos y mejores prácticas

### ✅ Buenas prácticas:

1. **Validar datos de entrada**: Asegúrate de que los valores sean razonables
   - Tamaño > 0
   - Habitaciones ≥ 1
   - Baños ≥ 1
   - Edad ≥ 0

2. **Mismo rango que entrenamiento**: El modelo funciona mejor con valores similares a los de entrenamiento
   - Si entrenaste con casas de 50-300 m², evita predecir para 500 m²

3. **Interpretar con cautela**: Recuerda que son estimaciones
   - El modelo tiene un error promedio (MAE)
   - Factores no incluidos pueden afectar el precio real (ubicación exacta, vistas, etc.)

### ⚠️ Limitaciones:

- El modelo solo considera las características numéricas entrenadas
- No incluye factores como: vecindario específico, vistas, acabados de lujo, etc.
- Es una **estimación estadística**, no un avalúo profesional